In [1]:
from collections import OrderedDict

import numpy as np
import pandas as pd
import pytest
from scipy.special import expit as inv_logit
from scipy.stats import bernoulli

from adopy import Engine, Model, Task

In [2]:
task = Task(
    name='Psychometric function estimation',
    designs=['stimulus'],
    responses=['choice'],
)

In [3]:
def func_logistic_log_lik(choice, stimulus,
                          guess_rate, lapse_rate, threshold, slope):
    f = inv_logit(slope * (stimulus - threshold))
    p = guess_rate + (1 - guess_rate - lapse_rate) * f
    return bernoulli.logpmf(choice, p)


model = Model(
    name='Logistic',
    task=task,
    params=['guess_rate', 'lapse_rate', 'threshold', 'slope'],
    func=func_logistic_log_lik,
)

In [4]:
grid_design = {
    'stimulus': np.linspace(20 * np.log10(.05), 20 * np.log10(400), 100)
}

grid_param = {
    'threshold': np.linspace(20 * np.log10(.1), 20 * np.log10(200), 100),
    'slope': np.linspace(0, 10, 101)[0:],
    'guess_rate': [0.5],
    'lapse_rate': [0.05],
}

grid_response = {'choice': [0, 1]}

In [5]:
%%time
engine = Engine(
    task=task,
    model=model,
    grid_design=grid_design,
    grid_param=grid_param,
    grid_response=grid_response,
)

CPU times: user 304 ms, sys: 49.2 ms, total: 353 ms
Wall time: 389 ms


In [6]:
%time d = engine.get_design()

CPU times: user 545 µs, sys: 29 µs, total: 574 µs
Wall time: 574 µs


In [7]:
%timeit d = engine.get_design()

135 µs ± 20 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [8]:
d = engine.get_design()

In [9]:
%time engine.update(d, 1)

CPU times: user 51.8 ms, sys: 3.34 ms, total: 55.2 ms
Wall time: 54.5 ms


In [10]:
%timeit engine.update(d, 1)

66.8 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
